In [1]:
# import packages
import os

import gym
import numpy as np
import math

In [2]:
# Import the environment we will use in this assignment
env=gym.make('CliffWalking-v0') 

# Show the model
print(f"Number of States {env.nS}, Number of Actions {env.nA}")
print(f"Reward range {env.reward_range}")

Number of States 48, Number of Actions 4
Reward range (-inf, inf)


In [3]:
env.reset() # reset the environment 

36

In [4]:
# visualize the current state
env.render()

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
x  C  C  C  C  C  C  C  C  C  C  T



In [5]:
# Q learning
class QLearningAgent(object):
    def __init__(self, state_n, act_n, learning_rate=0.01, gamma=0.9, epsilon=0.5):
        self.act_n = act_n
        self.lr = learning_rate
        self.gamma = gamma
        self.epsilon = epsilon
        self.Q = np.zeros((state_n, act_n))

    # explore by probabilty of epsilon
    def sample(self, state):
        if np.random.rand() < self.epsilon:
            action = np.random.choice(self.act_n)
        else:
            action = self.predict(state)
        return action

    # greedy
    def predict(self, state):
        Q_list = self.Q[state, :]
        maxQ = np.max(Q_list)
        action_list = np.where(Q_list == maxQ)[0]  # there may be many actions have same max Q value
        action = np.random.choice(action_list)
        return action

    # update Q-table
    def learn(self, state, action, reward, next_state, done):
        predict_Q = self.Q[state, action]
        if done:
            target_Q = reward
        else:
            target_Q = reward + self.gamma * np.max(self.Q[next_state, :]) # Q-learning
        self.Q[state, action] += self.lr * (target_Q - predict_Q)

In [6]:
# SARSA
class SarsaAgent(object):
    def __init__(self, state_n, act_n, learning_rate=0.01, gamma=0.9, epsilon=0.5):
        self.act_n = act_n
        self.lr = learning_rate
        self.gamma = gamma
        self.epsilon = epsilon
        self.Q = np.zeros((state_n, act_n))

    # explore by probabilty of epsilon
    def sample(self, state):
        if np.random.rand() < self.epsilon:
            action = np.random.choice(self.act_n)
        else:
            action = self.predict(state)
        return action

    # greedy
    def predict(self, state):
        Q_list = self.Q[state, :]
        maxQ = np.max(Q_list)
        action_list = np.where(Q_list == maxQ)[0]  # there may be many actions have same max Q value
        action = np.random.choice(action_list)
        return action

    # update Q-table
    def learn(self, state, action, reward, next_state, next_action, done):
        predict_Q = self.Q[state, action]
        if done:
            target_Q = reward
        else:
            target_Q = reward + self.gamma * self.Q[next_state, next_action] # SARSA
        self.Q[state, action] += self.lr * (target_Q - predict_Q)

In [7]:
# training episode of Q learning
def Q_run_episode(env, agent, render=False):
    # initialize the new episode
    total_steps = 0
    total_reward = 0

    state = env.reset()

    while True:
        action = agent.sample(state) # choose action by epsilon-greedy
        next_state, reward, done, _ = env.step(action) # observation
        agent.learn(state, action, reward, next_state, done) # update Q-table

        state = next_state
        total_reward += reward
        total_steps += 1
        if render:
            env.render()
        if done:
            break
    return total_reward, total_steps

# training episode of SARSA
def SARSA_run_episode(env, agent, render=False):
    # initialize the new episode
    total_steps = 0
    total_reward = 0

    state = env.reset()
    action = agent.sample(state) # choose action by epsilon-greedy
    
    while True:
        next_state, reward, done, _ = env.step(action) # observation
        next_action = agent.sample(next_state)   # get next action
        agent.learn(state, action, reward, next_state, next_action, done) # update Q-table

        action = next_action
        state = next_state
        total_reward += reward
        total_steps += 1
        if render:
            env.render()
        if done:
            break
    return total_reward, total_steps

In [8]:
# test episode
def test_episode(env, agent):
    total_reward = 0
    state = env.reset()
    while True:
        action = agent.predict(state) # greedy
        next_state, reward, done, _ = env.step(action)
        total_reward += reward
        state = next_state
        if done:
            break
    return total_reward

In [9]:
# training steps
def run_train(agent, max_episode, render_every_episode=50, 
              start_epsilon=1, end_epsilon=0.1, epsilon_decay_rate=100, mode="q-learning"):
    
    reward_history = []
    for episode in range(max_episode):
        if (episode+1) % render_every_episode == 0:
            render = True
        else:
            render = False
        agent.epsilon = end_epsilon + (start_epsilon - end_epsilon) * math.exp(-episode / epsilon_decay_rate)
        if mode == "q-learning":
            ep_reward, ep_steps = Q_run_episode(env, agent, render)
        elif mode == "SARSA":
            ep_reward, ep_steps = SARSA_run_episode(env, agent, render)
        else:
            raise ValueError('mode should be either "q-learning" or "SARSA"')
            
        print('Episode {}: steps = {} , reward = {}, epsilon = {}'.format(episode+1, ep_steps, ep_reward, agent.epsilon))
        reward_history.append(ep_reward)
        
    return reward_history 

# test steps
def run_test(agent, max_episode):
    test_result = []
    for episode in range(max_episode):
        test_reward = test_episode(env, agent)
        print('test episode {}: test reward = {}'.format(episode+1, test_reward))
        test_result.append(test_reward)
        if episode == max_episode - 1:
            print('average test reward = {}'.format(np.mean(test_result)))

In [10]:
# training parameters
max_episode = 1000
render_every_episode = 100
start_epsilon = 1
end_epsilon = 0.1
epsilon_decay_rate = 100

# make environment
env = gym.make("CliffWalking-v0")  # 0 up, 1 right, 2 down, 3 left

# make Q learning agent with hyperparameters
Q_agent = QLearningAgent(
    state_n=env.observation_space.n,
    act_n=env.action_space.n,
    learning_rate=0.1,
    gamma=0.9,
    epsilon=1)

# train Q learning for 1000 episodes and print rewards of every episode
Q_reward = run_train(Q_agent, max_episode, render_every_episode, 
              start_epsilon, end_epsilon, epsilon_decay_rate, mode="q-learning")

# check the results
run_test(Q_agent, max_episode=100)

Episode 1: steps = 15627 , reward = -159078, epsilon = 1.0
Episode 2: steps = 1117 , reward = -8641, epsilon = 0.9910448503742513
Episode 3: steps = 2693 , reward = -23978, epsilon = 0.9821788059760798
Episode 4: steps = 2851 , reward = -25522, epsilon = 0.9734009801936573
Episode 5: steps = 6505 , reward = -64420, epsilon = 0.9647104952370908
Episode 6: steps = 5671 , reward = -60715, epsilon = 0.9561064820506426
Episode 7: steps = 4026 , reward = -38874, epsilon = 0.9475880802258239
Episode 8: steps = 2077 , reward = -19798, epsilon = 0.9391544379153535
Episode 9: steps = 1508 , reward = -13586, epsilon = 0.9308047117479722
Episode 10: steps = 2856 , reward = -25923, epsilon = 0.9225380667441053
Episode 11: steps = 637 , reward = -4399, epsilon = 0.9143536762323635
Episode 12: steps = 1197 , reward = -9810, epsilon = 0.9062507217668754
Episode 13: steps = 316 , reward = -2296, epsilon = 0.8982283930454418
Episode 14: steps = 1595 , reward = -14861, epsilon = 0.8902858878285052
Episod

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  x  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  x  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  x  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  x  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  x  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  x  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o

Episode 201: steps = 19 , reward = -19, epsilon = 0.22180175491295145
Episode 202: steps = 23 , reward = -122, epsilon = 0.2205898072019245
Episode 203: steps = 29 , reward = -29, epsilon = 0.21938991857210954
Episode 204: steps = 30 , reward = -129, epsilon = 0.2182019690336438
Episode 205: steps = 13 , reward = -13, epsilon = 0.21702583979058332
Episode 206: steps = 15 , reward = -15, epsilon = 0.2158614132290238
Episode 207: steps = 17 , reward = -17, epsilon = 0.21470857290533868
Episode 208: steps = 21 , reward = -21, epsilon = 0.2135672035345349
Episode 209: steps = 19 , reward = -19, epsilon = 0.21243719097872418
Episode 210: steps = 44 , reward = -341, epsilon = 0.21131842223570935
Episode 211: steps = 16 , reward = -16, epsilon = 0.21021078542768373
Episode 212: steps = 33 , reward = -132, epsilon = 0.20911416979004352
Episode 213: steps = 38 , reward = -335, epsilon = 0.20802846566031108
Episode 214: steps = 13 , reward = -13, epsilon = 0.2069535644671687
Episode 215: steps =

Episode 344: steps = 15 , reward = -15, epsilon = 0.12914824669561634
Episode 345: steps = 18 , reward = -117, epsilon = 0.1288582167950747
Episode 346: steps = 14 , reward = -14, epsilon = 0.12857107274026114
Episode 347: steps = 13 , reward = -13, epsilon = 0.12828678581653094
Episode 348: steps = 13 , reward = -13, epsilon = 0.12800532759495478
Episode 349: steps = 14 , reward = -14, epsilon = 0.12772666992947598
Episode 350: steps = 14 , reward = -14, epsilon = 0.12745078495409573
Episode 351: steps = 15 , reward = -15, epsilon = 0.12717764508008667
Episode 352: steps = 15 , reward = -15, epsilon = 0.1269072229932337
Episode 353: steps = 19 , reward = -19, epsilon = 0.1266394916511028
Episode 354: steps = 31 , reward = -130, epsilon = 0.12637442428033668
Episode 355: steps = 15 , reward = -15, epsilon = 0.12611199437397735
Episode 356: steps = 16 , reward = -16, epsilon = 0.1258521756888155
Episode 357: steps = 15 , reward = -15, epsilon = 0.12559494224276607
Episode 358: steps = 1

o  o  o  o  x  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  x  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  x  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  x  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  x  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
x  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
x  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  x  o  o  o

Episode 671: steps = 26 , reward = -125, epsilon = 0.10110782071240614
Episode 672: steps = 24 , reward = -123, epsilon = 0.10109679771214158
Episode 673: steps = 24 , reward = -123, epsilon = 0.10108588439256225
Episode 674: steps = 15 , reward = -15, epsilon = 0.10107507966232707
Episode 675: steps = 13 , reward = -13, epsilon = 0.10106438244095402
Episode 676: steps = 13 , reward = -13, epsilon = 0.10105379165871206
Episode 677: steps = 25 , reward = -124, epsilon = 0.10104330625651414
Episode 678: steps = 17 , reward = -17, epsilon = 0.10103292518581129
Episode 679: steps = 13 , reward = -13, epsilon = 0.10102264740848779
Episode 680: steps = 24 , reward = -222, epsilon = 0.10101247189675736
Episode 681: steps = 13 , reward = -13, epsilon = 0.10100239763306033
Episode 682: steps = 13 , reward = -13, epsilon = 0.10099242360996194
Episode 683: steps = 19 , reward = -19, epsilon = 0.10098254883005156
Episode 684: steps = 14 , reward = -14, epsilon = 0.10097277230584299
Episode 685: st

Episode 828: steps = 15 , reward = -15, epsilon = 0.10023047676213143
Episode 829: steps = 13 , reward = -13, epsilon = 0.10022818348003126
Episode 830: steps = 14 , reward = -113, epsilon = 0.10022591301646926
Episode 831: steps = 13 , reward = -13, epsilon = 0.10022366514439716
Episode 832: steps = 13 , reward = -13, epsilon = 0.1002214396390259
Episode 833: steps = 19 , reward = -118, epsilon = 0.10021923627780306
Episode 834: steps = 28 , reward = -127, epsilon = 0.10021705484039072
Episode 835: steps = 13 , reward = -13, epsilon = 0.10021489510864327
Episode 836: steps = 26 , reward = -224, epsilon = 0.10021275686658578
Episode 837: steps = 13 , reward = -13, epsilon = 0.10021063990039225
Episode 838: steps = 16 , reward = -115, epsilon = 0.10020854399836429
Episode 839: steps = 13 , reward = -13, epsilon = 0.10020646895090995
Episode 840: steps = 21 , reward = -120, epsilon = 0.10020441455052276
Episode 841: steps = 22 , reward = -220, epsilon = 0.10020238059176097
Episode 842: s

o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  x  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  x  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  x  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  x  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
x  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
x  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
x  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C

In [11]:
# make SARSA agent with hyperparameters
SARSA_agent = SarsaAgent(
    state_n=env.observation_space.n,
    act_n=env.action_space.n,
    learning_rate=0.1,
    gamma=0.9,
    epsilon=1)

# train SARSA for 1000 episodes and print rewards of every episode
SARSA_reward = run_train(SARSA_agent, max_episode, render_every_episode, 
              start_epsilon, end_epsilon, epsilon_decay_rate, mode="SARSA")

# check the results
run_test(SARSA_agent, max_episode=100)

Episode 1: steps = 5149 , reward = -52867, epsilon = 1.0
Episode 2: steps = 7132 , reward = -68611, epsilon = 0.9910448503742513
Episode 3: steps = 7562 , reward = -63992, epsilon = 0.9821788059760798
Episode 4: steps = 2478 , reward = -23961, epsilon = 0.9734009801936573
Episode 5: steps = 3112 , reward = -28555, epsilon = 0.9647104952370908
Episode 6: steps = 157 , reward = -1444, epsilon = 0.9561064820506426
Episode 7: steps = 1943 , reward = -18476, epsilon = 0.9475880802258239
Episode 8: steps = 470 , reward = -2252, epsilon = 0.9391544379153535
Episode 9: steps = 1773 , reward = -12366, epsilon = 0.9308047117479722
Episode 10: steps = 137 , reward = -830, epsilon = 0.9225380667441053
Episode 11: steps = 140 , reward = -140, epsilon = 0.9143536762323635
Episode 12: steps = 1889 , reward = -14363, epsilon = 0.9062507217668754
Episode 13: steps = 297 , reward = -1683, epsilon = 0.8982283930454418
Episode 14: steps = 1234 , reward = -6184, epsilon = 0.8902858878285052
Episode 15: ste

x  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

x  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

x  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
x  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

x  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  x  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  x  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o

o  x  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  x  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
o  x  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  x  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

x  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

x  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  x  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

x  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o

o  C  C  C  C  C  C  C  C  C  C  T

o  o  x  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  x  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  x  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  x  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

x  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
x  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
x  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C

o  C  C  C  C  C  C  C  C  C  C  T

x  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

x  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
x  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
o  x  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  x  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

x  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  x  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C

...

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  x  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  x  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  x  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  x  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  x  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  x  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  x  o  o  o
o  o  o  o  o

o  o  o  o  o  o  o  o  o  o  o  o
x  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

x  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  x  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  x  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  x  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  x  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  x  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  x  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  x  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  x  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  x  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  x  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  x  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  x  o  o  o  o  o  o  o  o
o  o  o  o  o

o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  x
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  x

Episode 900: steps = 18 , reward = -18, epsilon = 0.10011218508391383
Episode 901: steps = 18 , reward = -18, epsilon = 0.10011106882367801
Episode 902: steps = 17 , reward = -17, epsilon = 0.10010996367041713
Episode 903: steps = 19 , reward = -19, epsilon = 0.10010886951361493
Episode 904: steps = 21 , reward = -21, epsilon = 0.10010778624385482
Episode 905: steps = 17 , reward = -17, epsilon = 0.10010671375280891
Episode 906: steps = 17 , reward = -17, epsilon = 0.10010565193322721
Episode 907: steps = 18 , reward = -18, epsilon = 0.10010460067892687
Episode 908: steps = 19 , reward = -19, epsilon = 0.1001035598847816
Episode 909: steps = 19 , reward = -19, epsilon = 0.1

test episode 20: test reward = -17
test episode 21: test reward = -17
test episode 22: test reward = -17
test episode 23: test reward = -17
test episode 24: test reward = -17
test episode 25: test reward = -17
test episode 26: test reward = -17
test episode 27: test reward = -17
test episode 28: test reward = -17
test episode 29: test reward = -17
test episode 30: test reward = -17
test episode 31: test reward = -17
test episode 32: test reward = -17
test episode 33: test reward = -17
test episode 34: test reward = -17
test episode 35: test reward = -17
test episode 36: test reward = -17
test episode 37: test reward = -17
test episode 38: test reward = -17
test episode 39: test reward = -17
test episode 40: test reward = -17
test episode 41: test reward = -17
test episode 42: test reward = -17
test episode 43: test reward = -17
test episode 44: test reward = -17
test episode 45: test reward = -17
test episode 46: test reward = -17
test episode 47: test reward = -17
test episode 48: tes

In [14]:
# save results
import csv
with open('data.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Q-learning", "SARSA"])
    for i in range(min(len(Q_reward),len(SARSA_reward))):
        writer.writerow([Q_reward[i],SARSA_reward[i]])